In [1]:
import numpy as np
import matplotlib.pyplot as pyplot
import pandas as pd
import scipy.sparse as sps
%matplotlib inline  
%load_ext Cython

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
import optuna

C:\Users\feder\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# data_train_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv"
# data_target_user_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv"
data_train_path="data_train.csv"
data_target_user_path="data_target_users_test.csv"
data_train = pd.read_csv(data_train_path)
data_target = pd.read_csv(data_target_user_path)

In [3]:
URM_all =  data_train.pivot(index='row', columns='col', values='data').fillna(0)
item_map = {i : item for i, item in enumerate(URM_all.columns)}
user_map = {i : user for i, user in enumerate(data_target["user_id"])}
item_map_inv = {item : i for i, item in item_map.items()}
user_map_inv = {user : i for i, user in user_map.items()}
missing_index = [x for x in range(1,13025) if x not in URM_all.index.tolist()]
add_urm = pd.DataFrame(index = missing_index, columns = URM_all.columns).fillna(0)
URM_all = pd.concat([URM_all, add_urm]).sort_index()
del add_urm
del missing_index
#data_target["user_id"] = data_target["user_id"]
URM_all = URM_all.to_numpy()
URM_all = sps.csr_matrix(URM_all)
URM_all

<13024x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [4]:
test_folds = 3

URM_trains = [None] * test_folds
evaluator_tests = [None] * test_folds
for i in range(0, test_folds):
    URM_trains[i], URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
    evaluator_tests[i] = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2587 (19.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2492 (19.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2558 (19.6%) Users that have less than 1 test interactions


In [11]:
def train_evaluate(optuna_trial):
    topk = optuna_trial.suggest_int("topK", 10, 100)
    alpha = optuna_trial.suggest_float("alpha", 0.5, 0.9)

    mAP = 0.0
    for i in range(0, test_folds):
        P3alpha = P3alphaRecommender(URM_trains[i], verbose=False)
        P3alpha.fit(topK=topk, alpha=alpha)
        result_df, _ = evaluator_tests[i].evaluateRecommender(P3alpha)
        mAP += result_df["MAP"].values[0]
    mAP /= test_folds
    return mAP

In [13]:
study = optuna.create_study(direction="maximize")
study.optimize(train_evaluate, n_trials=400)

[I 2023-12-11 15:01:15,287] A new study created in memory with name: no-name-3d2faf75-3a06-404e-96d0-6a64a6ef8e01


EvaluatorHoldout: Processed 10437 (100.0%) in 8.28 sec. Users per second: 1261
EvaluatorHoldout: Processed 10532 (100.0%) in 8.38 sec. Users per second: 1256
EvaluatorHoldout: Processed 10466 (100.0%) in 7.56 sec. Users per second: 1385


[I 2023-12-11 15:02:04,254] Trial 0 finished with value: 0.03491877556910843 and parameters: {'topK': 87, 'alpha': 0.885689947306503}. Best is trial 0 with value: 0.03491877556910843.


EvaluatorHoldout: Processed 10437 (100.0%) in 7.27 sec. Users per second: 1436
EvaluatorHoldout: Processed 10532 (100.0%) in 7.29 sec. Users per second: 1444
EvaluatorHoldout: Processed 10466 (100.0%) in 7.29 sec. Users per second: 1435


[I 2023-12-11 15:02:47,100] Trial 1 finished with value: 0.03270082096509213 and parameters: {'topK': 62, 'alpha': 0.5478382689790888}. Best is trial 0 with value: 0.03491877556910843.


...

[I 2023-12-11 19:25:10,060] Trial 397 finished with value: 0.0352472898985953 and parameters: {'topK': 59, 'alpha': 0.8641241974775686}. Best is trial 307 with value: 0.04241910694159847.


EvaluatorHoldout: Processed 10437 (100.0%) in 7.16 sec. Users per second: 1458
EvaluatorHoldout: Processed 10532 (100.0%) in 7.05 sec. Users per second: 1493
EvaluatorHoldout: Processed 10466 (100.0%) in 7.09 sec. Users per second: 1476


[I 2023-12-11 19:25:49,669] Trial 398 finished with value: 0.04234504306492983 and parameters: {'topK': 39, 'alpha': 0.753255039602179}. Best is trial 307 with value: 0.04241910694159847.


EvaluatorHoldout: Processed 10437 (100.0%) in 6.85 sec. Users per second: 1523
EvaluatorHoldout: Processed 10532 (100.0%) in 7.12 sec. Users per second: 1478
EvaluatorHoldout: Processed 10466 (100.0%) in 7.00 sec. Users per second: 1494


[I 2023-12-11 19:26:28,850] Trial 399 finished with value: 0.033216821081010574 and parameters: {'topK': 43, 'alpha': 0.5058142481763646}. Best is trial 307 with value: 0.04241910694159847.


In [19]:
# [I 2023-12-11 10:49:16,403] Trial 65 finished with value: 0.042232405327327555 and parameters: {'topK': 39, 'alpha': 0.7288366732392773}. Best is trial 65 with value: 0.042232405327327555.
study.get_trials()

[FrozenTrial(number=0, state=1, values=[0.03491877556910843], datetime_start=datetime.datetime(2023, 12, 11, 15, 1, 15, 289775), datetime_complete=datetime.datetime(2023, 12, 11, 15, 2, 4, 254521), params={'topK': 87, 'alpha': 0.885689947306503}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'topK': IntDistribution(high=100, log=False, low=10, step=1), 'alpha': FloatDistribution(high=0.9, log=False, low=0.5, step=None)}, trial_id=0, value=None),
 FrozenTrial(number=1, state=1, values=[0.03270082096509213], datetime_start=datetime.datetime(2023, 12, 11, 15, 2, 4, 255515), datetime_complete=datetime.datetime(2023, 12, 11, 15, 2, 47, 100373), params={'topK': 62, 'alpha': 0.5478382689790888}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'topK': IntDistribution(high=100, log=False, low=10, step=1), 'alpha': FloatDistribution(high=0.9, log=False, low=0.5, step=None)}, trial_id=1, value=None),
 FrozenTrial(number=2, state=1, values=[0.0388957

In [ ]:
# params={'topK': 38, 'alpha': 0.737157602088028} values=[0.04241910694159847]